# Text Extraction
***

In [ ]:
import numpy as np
import cv2
from PIL import Image
from utilities.text_extraction.detect_roi import detect_roi
from utilities.text_extraction.timestamp_extraction import extract_timestamps_from_image
from utilities.text_extraction.preprocessing import preprocess_image_for_tesseract

image = Image.open(r"testing\assets\example_rois\example_roi_1.png")
arr = np.array(image)

results = extract_timestamps_from_image(arr, print_results=True, preprocessing_func=None)
print("quarter: ", results.quarter)
print("time_remaining: ", results.time_remaining)

### OCR-Testing 

In [1]:
import os
import time
import numpy as np
from PIL import Image

from utilities.text_extraction.timestamp_extraction import *
from utilities.text_extraction.preprocessing import *


def test_ocr_benchmark():
    """Tests text-extraction on a set of benchmark images. Returns an accuracy score as a [0-1] float."""

    path_to_test_images = r"testing\assets\example_rois"
    images = [path for path in os.listdir(path_to_test_images)]

    start_time = time.time()
    found, total = 0, 0
    for path in images:
        img_path = os.path.join(path_to_test_images, path)
        image = Image.open(img_path)
        img_arr = np.array(image)
        result = extract_timestamps_from_image(
            img_arr, preprocessing_func=preprocess_image_for_tesseract)
        print(result)
        exit()
        if result.time_remaining and result.quarter:
            found += 1
        total += 1


    print("%s seconds" % ((time.time() - start_time) / total), " / image")
    print(found, "/",  total)
    return found / total

test_ocr_benchmark()

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


TypeError: extract_text_from_image_with_easyocr() got an unexpected keyword argument 'print_results'

In [ ]:
import os
for file in os.listdir(r"testing\assets\example_rois"):
    if file[-3: ] == 'PNG':
        os.rename(os.path.join(r"testing\assets\example_rois", file), os.path.join(r"testing\assets\example_rois", file).strip('PNG') + "png")


In [ ]:
import os
import time
import numpy as np
from PIL import Image
from utilities.text_extraction.timestamp_extraction import extract_timestamps_from_images
from utilities.text_extraction.preprocessing import preprocess_image_for_tesseract



total = 0
found = 0
for game in range (1, 31):
    try:
        quarter_img = np.array(Image.open(
            rf"testing\assets\example_cropped_rois\quarter\quarter_{game}.PNG"))
        time_remaining_img = np.array(Image.open(
            rf"testing\assets\example_cropped_rois\time_remaining\time_remaining_{game}.PNG"))
        roi = extract_timestamps_from_images(
            quarter_img, time_remaining_img, preprocessing_func=preprocess_image_for_tesseract)
        if roi.quarter and roi.time_remaining:
            found += 1
        total += 1
    except:
        pass


print(found, "/", total)
